In [ ]:
%%writefile nltk_utils.py
import nltk
import numpy as np



def tokenize(sentence):

    tokens = nltk.word_tokenize(sentence)


    return [w.lower() for w in tokens]

def encode_sentence(sentence, vocab_dict):
    encoded = []
    for word in sentence:
        if word in vocab_dict:
            encoded.append(vocab_dict[word])
        else:
            encoded.append(vocab_dict["<UNKN>"])
    return encoded

def pad_seq(batch, max_len):
    padded_batch = []
    for seq in batch:
        current_len = len(seq)
        if current_len < max_len:
            diff = max_len - current_len
            zeros = [0] * diff
            new_seq = seq + zeros
            padded_batch.append(new_seq)
        else:
            new_seq = seq[:max_len]
            padded_batch.append(new_seq)
    return padded_batch

Overwriting nltk_utils.py


In [ ]:
%%writefile model.py
# model.py

import torch
import torch.nn as nn
class RNNNet(nn.Module):
  def __init__(self,vocab_size,embed_dim,hidden_size,num_classes):

    super(RNNNet,self).__init__()


    self.embed=nn.Embedding(vocab_size,embed_dim)



    self.lstm=nn.LSTM(embed_dim,hidden_size,batch_first=True)




    self.fc=nn.Linear(hidden_size,num_classes)




  def forward(self,x):


    out=self.embed(x)


    out,_=self.lstm(out)


    out=out[:,-1,:]



    out=self.fc(out)



    return out




Overwriting model.py


In [ ]:
%%writefile train.py
import torch
import numpy as np
import torch.nn as nn
from nltk_utils import tokenize,encode_sentence,pad_seq
from model import RNNNet
from torch.utils.data import Dataset,DataLoader
import random
import json



with open('intents.json','r') as f:
  intents=json.load(f)


vocab={"<PAD>":0,"<UNKN>":1}
current_index=2

tags=[]
xy=[]

for intent in intents['intents']:
  tag=intent['tag']
  tags.append(tag)


  for pattern in intent['patterns']:
    w=tokenize(pattern)

    xy.append((w,tag))


    for word in w:
      if word not in vocab:
        vocab[word]=current_index
        current_index+=1





tags = sorted(set(tags))

max_len=10

X_train=[]
y_train=[]


for (pattern_sentence,tag) in xy:
  encoded_sentence=encode_sentence(pattern_sentence,vocab)
  X_train.append(encoded_sentence)
  label=tags.index(tag)


  y_train.append(label)


X_train=pad_seq(X_train,max_len)

X_train=np.array(X_train)
y_train=np.array(y_train)




class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples=len(X_train)
    self.x_data=X_train
    self.y_data=y_train

  def __getitem__(self,index):
    return self.x_data[index],self.y_data[index]

  def __len__(self):
    return self.n_samples





batch_size=8
hidden_size=8
embed_dim=8
output_size=len(tags)
vocab_size=len(vocab)
learning_rate=0.001
num_epochs=2000



dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RNNNet(vocab_size, embed_dim, hidden_size, output_size).to(device)



criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)




for epoch in range(num_epochs):
    for (words, labels) in train_loader:


        words = words.to(dtype=torch.long).to(device)
        labels = labels.to(dtype=torch.long).to(device)


        outputs = model(words)


        loss = criterion(outputs, labels)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'Final Loss: {loss.item():.4f}')


data = {
    "model_state": model.state_dict(),
    "vocab_size": vocab_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "embed_dim": embed_dim,
    "vocab_dict": vocab,
    "tags": tags,
    "max_len": max_len
}

FILE = "data.pth"
torch.save(data, FILE)
print(f'Training complete. file saved to {FILE}')








Overwriting train.py


In [ ]:
!python train.py

Epoch [100/2000], Loss: 1.6237
Epoch [200/2000], Loss: 0.5017
Epoch [300/2000], Loss: 0.4666
Epoch [400/2000], Loss: 0.2759
Epoch [500/2000], Loss: 0.0726
Epoch [600/2000], Loss: 0.0644
Epoch [700/2000], Loss: 0.0388
Epoch [800/2000], Loss: 0.0606
Epoch [900/2000], Loss: 0.0211
Epoch [1000/2000], Loss: 0.0478
Epoch [1100/2000], Loss: 0.0091
Epoch [1200/2000], Loss: 0.0727
Epoch [1300/2000], Loss: 0.0032
Epoch [1400/2000], Loss: 0.0037
Epoch [1500/2000], Loss: 0.0053
Epoch [1600/2000], Loss: 0.0042
Epoch [1700/2000], Loss: 0.0198
Epoch [1800/2000], Loss: 0.0023
Epoch [1900/2000], Loss: 0.0015
Epoch [2000/2000], Loss: 0.0009
Final Loss: 0.0009
Training complete. file saved to data.pth
